In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

%load_ext dotenv
%dotenv

In [3]:
from sailor import RouteGenConfig, SailorDataWarehouse

context="ERP admin panel"

_config = RouteGenConfig.from_env()
_warehouse = SailorDataWarehouse(_config, context, db_path="../db")

routes = await _warehouse.create_routes(count=20)
sessions_group = await _warehouse.create_sessions(routes, count=200)

In [4]:
from typing import List
from sklearn.model_selection import train_test_split
from sailor.types.route_specs import SessionSpec

train_sessions: List[SessionSpec] = []
test_sessions: List[SessionSpec] = []

for _sessions in sessions_group:
    if len(_sessions) <= 1:
        train_sessions.extend(_train)
        continue

    _train, _test = train_test_split(_sessions, shuffle=True, test_size=0.2, random_state=14)
    train_sessions.extend(_train)
    test_sessions.extend(_test)

print(f"Train sessions: {len(train_sessions)}")
print(f"Test sessions: {len(test_sessions)}")

Train sessions: 3200
Test sessions: 800


In [5]:
from sailor.sailor_engine import KNNSailorEngine, SVCSailorEngine

knn_engine = KNNSailorEngine()
knn_engine.fit(routes, train_sessions)

svc_engine = SVCSailorEngine()
svc_engine.fit(routes, train_sessions)

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('svc', LinearSVC(class_weight='balanced', max_iter=2000))])

In [6]:
from sailor.sailor_engine import SailorEngine
from sklearn.metrics import classification_report, top_k_accuracy_score

top_k_score = 5
test_query = "Edit user"

def pre_validate_model(model: SailorEngine, sessions: List[SessionSpec]):
    print(f"Validating {model.__class__.__name__}...")

    test_queries = [s.context for s in sessions]
    targets = model.documentor.transform([s.target for s in sessions])
    target_names = [
        model.documentor.get_route(id).path
        for id in model.documentor.labels_
    ]

    return test_queries, targets, target_names

In [7]:
_test_queries, _targets, _target_names = pre_validate_model(knn_engine, test_sessions)

_predictions = knn_engine.pipeline.predict(_test_queries)
print(classification_report(_targets, _predictions, target_names=_target_names, zero_division=0))

_predictions_proba = knn_engine.pipeline.predict_proba(_test_queries)
_top_k_accuracy = top_k_accuracy_score(_targets, _predictions_proba, k=top_k_score)
print(f"Top-{top_k_score} accuracy: {_top_k_accuracy:.2f}")

Validating KNNSailorEngine...
                          precision    recall  f1-score   support

         /admin/invoices       0.97      0.70      0.81        40
   /admin/users/:id/edit       0.55      0.93      0.69        40
  /admin/invoices/create       0.80      0.82      0.81        40
         /admin/products       0.82      0.45      0.58        40
        /admin/customers       0.88      0.38      0.53        40
 /admin/inventory/adjust       0.70      0.70      0.70        40
     /admin/users/create       0.94      0.85      0.89        40
          /admin/reports       0.85      0.82      0.84        40
 /admin/settings/general       0.79      0.55      0.65        40
            /admin/users       0.76      0.47      0.58        40
        /admin/inventory       0.70      0.65      0.68        40
         /admin/settings       0.69      0.72      0.71        40
    /admin/customers/:id       0.63      0.97      0.76        40
       /admin/orders/:id       0.78      1.00

In [8]:
routes = knn_engine.predict(test_query)[:top_k_score]
for route in routes:
  print(f"Path: {route.path}, Score: {route.score:.2f}")
  print(f"Context: {route.context}")

Path: /admin/users/:id/edit, Score: 0.31
Context: admin users :id edit users edit erp admin update account modify user Edit user timezone Dorothy Lewis Modify user title Kevin Harris Modify user emergency contact Kimberly Clark Update user password Ashley Rodriguez Updat user details Julia Reed Modifiy user profile Willie Hill Update account information Brandon Jackson Update user infomation Modify user publications Virginia Turner Edit user information Bruce Peterson Edit user skills Billy Green Change user designation Albert Cooper Update user termination date Brandon Martin Modify profile details Linda Garcia Update user security question Sarah Robinson Edit user contact info Olivia Taylor Edit user interests Alice Evans Edit user patents Philip Phillips Updating username password Modifi user skills Eric Green Update user role Grace Griffin Alter user department Arthur Richardson Update user security questions Anthony Robinson Modifying user roles ERP Modify user department Rebecca 

In [9]:
_test_queries, _targets, _target_names = pre_validate_model(svc_engine, test_sessions)

_predictions = svc_engine.pipeline.predict(_test_queries)
report = classification_report(_targets, _predictions, target_names=_target_names, zero_division=0)
print(report)

_decisions = svc_engine.pipeline.decision_function(_test_queries)
_top_k_accuracy = top_k_accuracy_score(_targets, _decisions, k=top_k_score)

print(f"Top-{top_k_score} accuracy: {_top_k_accuracy:.2f}")

Validating SVCSailorEngine...
                          precision    recall  f1-score   support

         /admin/invoices       0.78      0.80      0.79        40
   /admin/users/:id/edit       0.58      0.93      0.71        40
  /admin/invoices/create       0.80      0.82      0.81        40
         /admin/products       0.74      0.42      0.54        40
        /admin/customers       0.88      0.35      0.50        40
 /admin/inventory/adjust       0.76      0.72      0.74        40
     /admin/users/create       0.87      0.85      0.86        40
          /admin/reports       0.85      0.82      0.84        40
 /admin/settings/general       0.67      0.55      0.60        40
            /admin/users       0.75      0.45      0.56        40
        /admin/inventory       0.78      0.62      0.69        40
         /admin/settings       0.66      0.57      0.61        40
    /admin/customers/:id       0.65      0.97      0.78        40
       /admin/orders/:id       0.78      1.00

In [10]:
routes = svc_engine.predict(test_query)[:top_k_score]
for route in routes:
  print(f"Path: {route.path}, Score: {route.score:.2f}")
  print(f"Context: {route.context}")

Path: /admin/users/:id/edit, Score: -0.05
Context: admin users :id edit users edit erp admin update account modify user Edit user timezone Dorothy Lewis Modify user title Kevin Harris Modify user emergency contact Kimberly Clark Update user password Ashley Rodriguez Updat user details Julia Reed Modifiy user profile Willie Hill Update account information Brandon Jackson Update user infomation Modify user publications Virginia Turner Edit user information Bruce Peterson Edit user skills Billy Green Change user designation Albert Cooper Update user termination date Brandon Martin Modify profile details Linda Garcia Update user security question Sarah Robinson Edit user contact info Olivia Taylor Edit user interests Alice Evans Edit user patents Philip Phillips Updating username password Modifi user skills Eric Green Update user role Grace Griffin Alter user department Arthur Richardson Update user security questions Anthony Robinson Modifying user roles ERP Modify user department Rebecca

In [11]:
_models_path = "../build/models"
knn_engine_path = knn_engine.save_model("knn_model", _models_path)
knn_model_size_mb = os.path.getsize(knn_engine_path)
print(f"Model saved to {knn_engine_path}, size: {knn_model_size_mb} MB")

svc_engine_path = svc_engine.save_model("svc_model", _models_path)
svc_model_size_mb = os.path.getsize(svc_engine_path)
print(f"Model saved to {svc_engine_path}, size: {svc_model_size_mb} MB")

Model saved to ../build/models/knn_model.pkl, size: 451162238 MB
Model saved to ../build/models/svc_model.pkl, size: 451300912 MB
